In [ ]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import skimage.draw

ABS_DIR = os.path.abspath("../../")
print(ABS_DIR)
sys.path.append(ABS_DIR)  # To find local version of the library

# Root directory of the project
#ROOT_DIR = os.path.abspath(ROOT_DIR+"/mrcnn")
ROOT_DIR = ABS_DIR
ROOT_DIR = os.path.join("~/Mask-RCNN Example/Mask_RCNN/mrcnn/")
print(ROOT_DIR)



from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
print(COCO_WEIGHTS_PATH)

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ABS_DIR, "logs")
print(DEFAULT_LOGS_DIR)
MODEL_DIR = os.path.join(ABS_DIR, "logs")

In [ ]:
class RiceConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "Rice Diseased"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + balloon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9


############################################################
#  Dataset
############################################################

class RiceDataset(utils.Dataset):
    
    def load_rice(self,ANNOTATED_DIR,IMAGE_DIR):
        """Load a subset of the Balloon dataset
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("diseased", 1, "diseased")
        
        # Train or validation dataset?
        #assert subset in ["train", "val"]
        #dataset_dir = os.path.join(dataset_dir, subset)
        #RICE_DIR = 
        #print(IMAGE_DIR)
        #print(ANNOTATED_DIR)
        JSONfiles = os.listdir(ANNOTATED_DIR)
        
        for file in JSONfiles:
            annotations =json.load(open(os.path.join(ANNOTATED_DIR,file)))
            annotations = list(annotations.values())
            #print(file)
            #print(annotations[2][0]['points'][0])
            shapes = annotations[2]
            #print(len(shapes))
            polygons = [r for r in shapes]
            #print(polygons)
            image_path = os.path.join(IMAGE_DIR, file[:-4]+'jpg')
            #print(image_path)
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]
            
            self.add_image(
                "diseased",
                image_id=file[:-4]+'jpg',  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons)
            
    def load_mask(self, image_id):
        """Generate instance masks for an image.
        Returns:
        masks: A bool array of shape [height, width, instance count] with
        one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        if image_info["source"] != "diseased":
            return super(self.__class__, self).load_mask(image_id)
        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        #print(info)
        #print(info['polygons'])
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        
        for i,p in enumerate(info["polygons"]):
            #print(i)
            #print(p['points'])
            all_x =[]
            all_y =[]
            for x,y in p['points']:
                #print(str(x)+" "+str(y))
                all_x.append(x)
                all_y.append(y)
            rr, cc = skimage.draw.polygon(all_y,all_x)
            mask[rr, cc, i] = 1
            all_x =[]
            all_y =[]
        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)
        
    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "diseased":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


def train(model):
    """Train the model."""
    # Training dataset.
    ANNOTATED_DIR = os.path.join(BASE_DIR, "datasets/rice/train/data_annotated/")
    IMAGE_DIR = os.path.join(BASE_DIR, "datasets/rice/train/JPEGImages/")
    dataset_train = RiceDataset()
    dataset_train.load_rice(ANNOTATED_DIR,IMAGE_DIR)
    dataset_train.prepare()

    # Validation dataset
    ANNOTATED_DIR = os.path.join(BASE_DIR, "datasets/rice/val/data_annotated/")
    IMAGE_DIR = os.path.join(BASE_DIR, "datasets/rice/val/JPEGImages/")
    dataset_val = RiceDataset()
    dataset_val.load_rice(ANNOTATED_DIR,IMAGE_DIR)
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=40,
                layers='head') 


def color_splash(image, mask):
    """Apply color splash effect.
    image: RGB image [height, width, 3]
    mask: instance segmentation mask [height, width, instance count]

    Returns result image.
    """
    # Make a grayscale copy of the image. The grayscale copy still
    # has 3 RGB channels, though.
    gray = skimage.color.gray2rgb(skimage.color.rgb2gray(image)) * 255
    # Copy color pixels from the original color image where mask is set
    if mask.shape[-1] > 0:
        # We're treating all instances as one, so collapse the mask into one layer
        mask = (np.sum(mask, -1, keepdims=True) >= 1)
        splash = np.where(mask, image, gray).astype(np.uint8)
    else:
        splash = gray.astype(np.uint8)
    return splash


def detect_and_color_splash(model, image_path=None, video_path=None):
    assert image_path or video_path

    # Image or video?
    if image_path:
        # Run model detection and generate the color splash effect
        print("Running on {}".format(args.image))
        # Read image
        image = skimage.io.imread(args.image)
        # Detect objects
        r = model.detect([image], verbose=1)[0]
        # Color splash
        splash = color_splash(image, r['masks'])
        # Save output
        file_name = "splash_{:%Y%m%dT%H%M%S}.png".format(datetime.datetime.now())
        skimage.io.imsave(file_name, splash)
    elif video_path:
        import cv2
        # Video capture
        vcapture = cv2.VideoCapture(video_path)
        width = int(vcapture.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(vcapture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = vcapture.get(cv2.CAP_PROP_FPS)

        # Define codec and create video writer
        file_name = "splash_{:%Y%m%dT%H%M%S}.avi".format(datetime.datetime.now())
        vwriter = cv2.VideoWriter(file_name,
                                  cv2.VideoWriter_fourcc(*'MJPG'),
                                  fps, (width, height))

        count = 0
        success = True
        while success:
            print("frame: ", count)
            # Read next image
            success, image = vcapture.read()
            if success:
                # OpenCV returns images as BGR, convert to RGB
                image = image[..., ::-1]
                # Detect objects
                r = model.detect([image], verbose=0)[0]
                # Color splash
                splash = color_splash(image, r['masks'])
                # RGB -> BGR to save image to video
                splash = splash[..., ::-1]
                # Add image to video writer
                vwriter.write(splash)
                count += 1
        vwriter.release()
    print("Saved to ", file_name)

In [ ]:
BASE_DIR = os.path.abspath("../../")
print(BASE_DIR)


ANNOTATED_DIR = os.path.join(BASE_DIR, "datasets/rice/train/data_annotated/")
IMAGE_DIR = os.path.join(BASE_DIR, "datasets/rice/train/JPEGImages/")
dataset_train = RiceDataset()
dataset_train.load_rice(ANNOTATED_DIR,IMAGE_DIR)
dataset_train.prepare()
print("Image Count: {}".format(len(dataset_train.image_ids)))
print("Class Count: {}".format(dataset_train.num_classes))
for i, info in enumerate(dataset_train.class_info):
    print("{:3}. {:50}".format(i, info['name']))

In [ ]:
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [ ]:
# Load random image and mask.
image_id = random.choice(dataset_train.image_ids)
image = dataset_train.load_image(image_id)
mask, class_ids = dataset_train.load_mask(image_id)
# Compute Bounding box
bbox = utils.extract_bboxes(mask)

# Display image and additional stats
print("image_id ", image_id, dataset_train.image_reference(image_id))
log("image", image)
log("mask", mask)
log("class_ids", class_ids)
log("bbox", bbox)
# Display image and instances
visualize.display_instances(image, bbox, mask, class_ids, dataset_train.class_names)

In [ ]:
BASE_DIR = os.path.abspath("../../")
print(BASE_DIR)

ANNOTATED_DIR = os.path.join(BASE_DIR, "datasets/rice/val/data_annotated/")
IMAGE_DIR = os.path.join(BASE_DIR, "datasets/rice/val/JPEGImages/")
dataset_val = RiceDataset()
dataset_val.load_rice(ANNOTATED_DIR,IMAGE_DIR)
dataset_val.prepare()
print("Image Count: {}".format(len(dataset_val.image_ids)))
print("Class Count: {}".format(dataset_val.num_classes))
for i, info in enumerate(dataset_val.class_info):
    print("{:3}. {:50}".format(i, info['name']))

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [1]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')

NameError: name 'model' is not defined

In [ ]:

# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
model.keras_model.save_weights(model_path)